<a href="https://colab.research.google.com/github/prodonn/public/blob/main/calcul_dose_de_radiation_re%C3%A7ue_git_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test de la fonction de calcul de la dose reçue en chaque point du tableau des contours

Source : Repertoire h3dmed/dicom

## Setup

In [ ]:
import os
import datetime
import glob

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['axes.grid'] = False

Détection du repertoire source contenant les inforamtions issues des dicoms suivant l'environnement de travail.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB :
  from google.colab import drive
  drive.mount('/drive')
  drive_dir = '/drive/MyDrive'
  data_dir = drive_dir + '/h3dmed/dicom'
  extract_dir = drive_dir + '/h3dmed/extract_dicom' 
else :
  data_dir ='../dicom_dir'
  extract_dir ='../extract_dicom'

Mounted at /drive


## Load the dataframes :

1.   df_dwell : dataframe contenant les positions des points d'arrêt et leurs temps correspondant
2.   df_contour : dataframe contenant les contours des différents parties du corps et des applicateurs.



In [ ]:
df_dwell = pd.read_csv(f'{extract_dir}/df_arrets_fantome_20220316.csv', index_col=0)
df_contour = pd.read_csv(f'{extract_dir}/df_contours_fantome_20220317.csv', index_col=0)

### Filtre sur les contours des applicateurs

In [ ]:
l_noms_contours = df_contour['ROIName'].unique()

In [ ]:
l_noms_contours

array(['Applicateur1', 'Applicateur2', 'Applicateur3', 'Applicateur4',
       'Applicateur5', 'Applicateur6', 'CONTOUR EXTERNE', 'CTVHR',
       'CTVRI', 'OeilDetG', 'Peau3mm', 'aide', 'peausaine',
       'Applicateur7', 'Applicateur8', 'Applicateur9', 'Applicateur10',
       'Applicateur11', 'Applicateur12'], dtype=object)

Peut t'on enlever les Applicateurs ?
Qu'est ce que l'aide ?

### On garde les contours hors Applicateur

In [ ]:
l_human_contours = [c for c in l_noms_contours if 'Applicateur' not in c ]
l_human_contours

['CONTOUR EXTERNE',
 'CTVHR',
 'CTVRI',
 'OeilDetG',
 'Peau3mm',
 'aide',
 'peausaine']

In [ ]:
df_human_contours = df_contour.loc[df_contour['ROIName'].isin(l_human_contours)].copy()

In [ ]:
df_human_contours['ROIName'].value_counts()

CONTOUR EXTERNE    185154
Peau3mm             79518
aide                76982
peausaine           73640
OeilDetG             3516
CTVRI                1928
CTVHR                1202
Name: ROIName, dtype: int64

### Creation du tableau des sources

In [ ]:
# Creation du tableau des sources 
sources_array = df_dwell[['x', 'y', 'z', 'CumulativeTimeWeight']].to_numpy()

## Définition de la fonction de calcul de radiation ! 

In [ ]:
import math

def exposure_rate(target, sources):
  #Fonction permettant de calculer une dose recue par un point target 
  #En entrée data_source, un tableau avec comme trois premieres colonnes les coord du point d'arret
  #Comme dernière colonne le temps d'arret associé 
  #target :  les coordonnées du point target. 
  #En sortie la dose recue par target en Gy
  dose_elementaire = 0
  # Coordonnées de la cible target
  target = np.array(target)
  # Initialisation des constantes 
  Gamma=4.69
  Fmed=0.971
  Ka=1.59
  Kb=1.23
  Mu=0.092  
  Activite_mCi=10
  data_source = sources
  for source in data_source :
    coord_source = source[:3]
    # Estimation de la distance source target
    r = np.sqrt(np.sum(np.array(coord_source-target)**2))

    h2oRatio=(1+Ka*(Mu*r)**Kb)*math.exp(-Mu*r)
    dose_elementaire = dose_elementaire + 10*Gamma*(Activite_mCi/(r**2))*(source[3]/3600)*Fmed*h2oRatio

  return dose_elementaire



In [ ]:
df_dwell.shape

(120, 6)

## Calcul de la radiation sur un échantillon, le CTVRI

In [ ]:
df_CTVRI_contours = df_contour.loc[df_contour['ROIName']=='CTVRI'].copy()

In [ ]:
df_CTVRI_contours.head(2)

,x,y,z,ReferencedROINumber,ROINumber,ROIName
186417,-7.65,6.47,-34.0,3.0,3.0,CTVRI
186418,-6.51,6.49,-34.0,3.0,3.0,CTVRI


In [ ]:
import time
df_CTVRI_contours['dose (Gy)'] = df_CTVRI_contours[['x','y','z']].apply(lambda coord: exposure_rate(coord, sources_array), axis=1)



In [ ]:
df_CTVRI_contours.head(2)

,x,y,z,ReferencedROINumber,ROINumber,ROIName,dose (Gy)
186417,-7.65,6.47,-34.0,3.0,3.0,CTVRI,0.745985
186418,-6.51,6.49,-34.0,3.0,3.0,CTVRI,0.709725


In [ ]:
df_CTVRI_contours.to_csv(f'{extract_dir}/df_dose_CTVRI.csv')

## Calcul de la radiation sur un échantillon, l'oeil

In [ ]:
df_OeilDetG_contours = df_contour.loc[df_contour['ROIName']=='OeilDetG'].copy()

In [ ]:
df_OeilDetG_contours['dose (Gy)'] = df_OeilDetG_contours[['x','y','z']].apply(lambda coord: exposure_rate(coord, sources_array), axis=1)

In [ ]:
df_OeilDetG_contours.head(2)

,x,y,z,ReferencedROINumber,ROINumber,ROIName,dose (Gy)
188345,-15.58,22.55,-23.0,5.0,5.0,OeilDetG,0.580935
188346,-15.48,22.67,-23.0,5.0,5.0,OeilDetG,0.571514
